In [ ]:
# command to download data directly from kaggle 
!kaggle datasets download -d  yelp-dataset/yelp-dataset/ -p ./data/raw_data/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import spacy
from pandas.io.json import json_normalize
import json
import ast

In [2]:
df_reviews = []

chunks = pd.read_json('./data/raw_data/yelp_academic_dataset_review.json', lines=True, chunksize=1000)
for chunk in tqdm(chunks):
    chunk_df = pd.DataFrame(chunk)
    chunk_df = chunk_df.sample(10, random_state=62)
    df_reviews.append(chunk_df)

df_reviews = pd.concat(df_reviews)

8022it [02:40, 50.05it/s]


In [3]:
df_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
846,rlewaaIzJ3n7dqLJa87yvQ,Hv_q_ZnSIoZwdcoH0CyV2Q,1SE9f6TEV5XhnTuuK01xwA,5,6,2,2,I've been a Wingstop fan for quite some time n...,2013-10-03 13:42:56
536,OQ48kD9XzF2xEGUufgKxow,xCokEpVNJDysHCXSEFvKvA,QITVudxg4pCScJnEUw46xg,5,0,0,0,We had a great experience leasing a new Equino...,2018-01-21 05:10:42
926,ze-OZk9WOhEe4bbdsHEeqA,cMEtAiW60I5wE_vLfTxoJQ,62HlThvB9Rm2QFu_OdXe7w,4,0,0,0,Great restaurant in the Sheridan Square neighb...,2012-04-28 21:52:51
538,O9kninOPpLT9Zkd1Mq8d4Q,Q1iDQmp576GFmaGAYJ760g,UFVy-8WC7pDzvBUzOvdwzA,4,1,0,0,"Loved this place. Food was delicious, although...",2013-05-07 13:41:05
196,kNrFMqAAs0b3sswbJyyEgQ,055mHXHyW4wE3dwVSNMs9g,3-q92s_9lKhAH9OgqEg_Dg,5,1,0,0,Finding a solid spot to get a quality cut can ...,2018-03-15 19:38:03


In [4]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80220 entries, 846 to 8021082
Data columns (total 9 columns):
review_id      80220 non-null object
user_id        80220 non-null object
business_id    80220 non-null object
stars          80220 non-null int64
useful         80220 non-null int64
funny          80220 non-null int64
cool           80220 non-null int64
text           80220 non-null object
date           80220 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 6.1+ MB


In [5]:
df_reviews.to_pickle('data/reviews_1pctsample.pkl')

In [6]:
df_reviews['num_words'] = [len(review) for review in df_reviews.text]

In [7]:
df_business = pd.read_json('data/raw_data/yelp_academic_dataset_business.json', lines=True)
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [8]:
df_new = pd.merge(left=df_reviews, right=df_business[['business_id', 'stars']], how='left', on='business_id', suffixes=('_rating','_avg'))
df_new.head()

,review_id,user_id,business_id,stars_rating,useful,funny,cool,text,date,num_words,stars_avg
0,rlewaaIzJ3n7dqLJa87yvQ,Hv_q_ZnSIoZwdcoH0CyV2Q,1SE9f6TEV5XhnTuuK01xwA,5,6,2,2,I've been a Wingstop fan for quite some time n...,2013-10-03 13:42:56,1541,3.0
1,OQ48kD9XzF2xEGUufgKxow,xCokEpVNJDysHCXSEFvKvA,QITVudxg4pCScJnEUw46xg,5,0,0,0,We had a great experience leasing a new Equino...,2018-01-21 05:10:42,475,3.0
2,ze-OZk9WOhEe4bbdsHEeqA,cMEtAiW60I5wE_vLfTxoJQ,62HlThvB9Rm2QFu_OdXe7w,4,0,0,0,Great restaurant in the Sheridan Square neighb...,2012-04-28 21:52:51,328,3.5
3,O9kninOPpLT9Zkd1Mq8d4Q,Q1iDQmp576GFmaGAYJ760g,UFVy-8WC7pDzvBUzOvdwzA,4,1,0,0,"Loved this place. Food was delicious, although...",2013-05-07 13:41:05,325,3.0
4,kNrFMqAAs0b3sswbJyyEgQ,055mHXHyW4wE3dwVSNMs9g,3-q92s_9lKhAH9OgqEg_Dg,5,1,0,0,Finding a solid spot to get a quality cut can ...,2018-03-15 19:38:03,295,3.0


In [9]:
df_new['sentiment'] = df_new['stars_rating'].where(df_new.stars_rating > 3, 0).where(df_new.stars_rating <= 3, 1)

In [10]:
df_new.head()

,review_id,user_id,business_id,stars_rating,useful,funny,cool,text,date,num_words,stars_avg,sentiment
0,rlewaaIzJ3n7dqLJa87yvQ,Hv_q_ZnSIoZwdcoH0CyV2Q,1SE9f6TEV5XhnTuuK01xwA,5,6,2,2,I've been a Wingstop fan for quite some time n...,2013-10-03 13:42:56,1541,3.0,1
1,OQ48kD9XzF2xEGUufgKxow,xCokEpVNJDysHCXSEFvKvA,QITVudxg4pCScJnEUw46xg,5,0,0,0,We had a great experience leasing a new Equino...,2018-01-21 05:10:42,475,3.0,1
2,ze-OZk9WOhEe4bbdsHEeqA,cMEtAiW60I5wE_vLfTxoJQ,62HlThvB9Rm2QFu_OdXe7w,4,0,0,0,Great restaurant in the Sheridan Square neighb...,2012-04-28 21:52:51,328,3.5,1
3,O9kninOPpLT9Zkd1Mq8d4Q,Q1iDQmp576GFmaGAYJ760g,UFVy-8WC7pDzvBUzOvdwzA,4,1,0,0,"Loved this place. Food was delicious, although...",2013-05-07 13:41:05,325,3.0,1
4,kNrFMqAAs0b3sswbJyyEgQ,055mHXHyW4wE3dwVSNMs9g,3-q92s_9lKhAH9OgqEg_Dg,5,1,0,0,Finding a solid spot to get a quality cut can ...,2018-03-15 19:38:03,295,3.0,1


In [11]:
df_new.to_pickle('data/yelp_review1pct_business_merge.pkl')